In [29]:
!pip install groq python-dotenv numpy tqdm datasets

In [67]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [40]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.1, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [41]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello. What math problem would you like help with today?


#### GSM8K 데이터셋 확인해보기

In [33]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [34]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [ ]:
### 수정해도 됩니다!
import time

def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama-3.1-8b-instant",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        time.sleep(1.2)  # rate limit 방지
        
        question = dataset[i]["question"]
        # 정답 숫자 추출 (원본 데이터의 #### 뒤 숫자)
        correct_answer_str = dataset[i]["answer"].split('####')[-1].strip()
        correct_answer = float(re.findall(r'-?\d+(?:\.\d+)?', correct_answer_str.replace(",", ""))[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)

            raw_predicted = extract_final_answer(response)
            predicted_answer = None

            # 안전한 숫자 변환 (에러 발생 시 None으로 처리하여 멈춤 방지)
            if raw_predicted is not None:
                try:
                    # 콤마 제거 후 숫자 형태만 다시 한번 추출하여 float 변환
                    num_match = re.findall(r'-?\d+(?:\.\d+)?', str(raw_predicted).replace(",", ""))
                    if num_match:
                        predicted_answer = float(num_match[-1])
                except (ValueError, TypeError, IndexError):
                    predicted_answer = None
            
            # 정답 비교 (predicted_answer가 None인 경우 계산을 건너뛰어 TypeError 방지)
            is_correct = False
            if predicted_answer is not None:
                # 두 값의 차이가 1e-5 미만이면 정답으로 인정
                if abs(predicted_answer - correct_answer) < 1e-5:
                    is_correct = True
                    correct += 1
            
            total += 1
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            # 5개마다 진행 상황 출력
            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}] | Current Acc.: [{current_acc:.2%}]")
                
    return results, correct/total if total > 0 else 0

In [36]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [40]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [48]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")

 10%|█         | 5/50 [00:08<01:19,  1.78s/it]

Progress: [5/50] | Current Acc.: [80.00%]


 20%|██        | 10/50 [00:17<01:08,  1.70s/it]

Progress: [10/50] | Current Acc.: [70.00%]


 30%|███       | 15/50 [00:39<02:34,  4.42s/it]

...

Progress: [40/50] | Current Acc.: [80.00%]


 90%|█████████ | 45/50 [03:11<00:24,  4.94s/it]

Progress: [45/50] | Current Acc.: [80.00%]


100%|██████████| 50/50 [03:34<00:00,  4.30s/it]

Progress: [50/50] | Current Acc.: [80.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [ ]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = (
            "Instruction:\n"
            "Solve the following math problems by thinking step-by-step. "
            "Show your work clearly and provide the final answer as a number after '####'.\n"
    )
    for i in range(num_examples):
        idx = sampled_indices[i]
        cur_question = train_dataset[idx]['question']
        cur_answer = train_dataset[idx]['answer']

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"


    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

PROMPT = construct_CoT_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samsples=50
)
save_final_result(results, accuracy, "CoT_prompting_0.txt")

 10%|█         | 5/50 [00:08<01:20,  1.80s/it]

Progress: [5/50] | Current Acc.: [100.00%]


 20%|██        | 10/50 [00:18<01:13,  1.83s/it]

Progress: [10/50] | Current Acc.: [90.00%]


 30%|███       | 15/50 [00:27<01:03,  1.82s/it]

...

Progress: [40/50] | Current Acc.: [77.50%]


 90%|█████████ | 45/50 [01:38<00:17,  3.59s/it]

Progress: [45/50] | Current Acc.: [77.78%]


100%|██████████| 50/50 [01:51<00:00,  2.24s/it]

Progress: [50/50] | Current Acc.: [80.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [ ]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(num_examples: int = 5) -> str:
    train_dataset = gsm8k_train
    sampled_indices = random.sample(range(len(train_dataset)), num_examples)

    # 지시사항을 군더더기 없이 바꿉니다. 
    # 모델이 수다 떨지 않고 바로 계산에 집중하게 합니다.
    prompt = (
        "Solve the following math problem step-by-step.\n"
        "Instructions:\n"
        "- Show clear logical steps.\n"
        "- You must end with 'Final Answer: #### [numerical value]'.\n\n"
    )

    for idx in enumerate(sampled_indices):
        cur_question = train_dataset[idx]['question']
        cur_answer = train_dataset[idx]['answer']

        prompt += f"Q: {cur_question}\nA: {cur_answer}\n\n"

    # 실제 문제 입력 시 문구를 최대한 짧게 유지하여 모델의 집중력을 높입니다.
    prompt += "Task: Solve the problem below accurately.\n"
    prompt += "Question: {question}\nAnswer:"

    return prompt

In [48]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

PROMPT = construct_my_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_5.txt")

 10%|█         | 5/50 [00:43<08:24, 11.22s/it]

Progress: [5/50] | Current Acc.: [80.00%]


 20%|██        | 10/50 [02:01<10:16, 15.42s/it]

Progress: [10/50] | Current Acc.: [70.00%]


 30%|███       | 15/50 [03:11<08:17, 14.22s/it]

...

Progress: [40/50] | Current Acc.: [87.50%]


 90%|█████████ | 45/50 [10:11<01:10, 14.11s/it]

Progress: [45/50] | Current Acc.: [88.89%]


100%|██████████| 50/50 [11:23<00:00, 13.68s/it]

Progress: [50/50] | Current Acc.: [88.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!